# Kerasでのネットワーク実装

手作りのニューラルネットワーク専用のフレームワーク「Keras」で構築しよう。まず、必要なパッケージを追加しよう

In [1]:
import numpy as np

# 入力、結合、活性化の「レイヤ」
from keras.layers import Input, Dense, Activation

# モデル
from keras.models import Model

# 活性化関数
from keras.activations import sigmoid

## モデル構築

Kerasのモデルは「レイヤ」で構築する。ネットワークの各パーツにたいして、適するレイヤがある：

- Input: 入力レイヤ。入力の数を定義する
- Dense: 全結合レイヤ。前レイヤの出力と本レイヤの出力をすべてつながるり、荷重を与える
- Activation：活性レイヤ。前レイヤの出力の出力に活性化関数で処理する

In [2]:
def create_model():
    
    # 入力レイヤ
    x = Input(shape=(2), dtype=float)
    
    # 全結合レイヤ（荷重）出力が１つ
    w = Dense(units=1)(x)
    
    # 活性化レイヤ。活性化関数はsigmoid
    y = Activation(sigmoid)(w)
    
    # モデル
    model = Model(inputs=x, outputs=y)
    
    return model

In [3]:
# 確認しましょう
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 1)                 3         
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


上記の出力を解析してみると…

`input_1 (InputLayer)        [(None, 2)]               0 `

入力レイヤは入力の数は２つがある。`None`はデータの数を示している。ANDとORの課題で、データの数は４点があるが、現時点でまだ確定ではないので`None`として表示されている。入力レイヤは学習できるパラメータはないので、パラメータの数は0である。

`dense (Dense)               (None, 1)                 3`

全結合レイヤの出力は１つしかない。上記と同様、まだ確定されていないデータの数は`None`で表示。なお、パラメータの数は3となる（荷重２つとバイアス１つ）

`activation (Activation)     (None, 1)                 0`

最後に活性化関数があり、出力は１つです。前の出力を処理するだけで、学習パラメータがない。

In [4]:
# 中身を確認
print (model.get_weights())

[array([[-1.0010679],
       [-1.0432086]], dtype=float32), array([0.], dtype=float32)]


## 推論
このまま（学習なし）で推論をしてみると：

In [5]:
x = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], dtype=float)

y_pred = model.predict(x)
print(y_pred)

1/1 [==============================] - 0s 118ms/step
[[0.5       ]
 [0.26053137]
 [0.2687315 ]
 [0.114632  ]]


## 学習

推論はまったく当たらないので、学習させましょう！

まず、kerasでは、そのまま学習できない。その先、`create_model()`で作ったモデルを「コンパイル」しなければならない。コンパイルすることにて、モデルをGPUで実行できるようになる

In [6]:
model.compile(optimizer="sgd", loss="mean_squared_error")

上記の命令を解説すると：

- `optimizer`:「最適化方法」という意味であり、勾配関数のことを表している。`sgd`と`Adam`をよく使われている
- `loss`:損失関数のこと。課題により、適する損失関数がありが、とりあえず引き続きに「平均二乗誤差」を利用

なお、学習は、以下の命令で実装：

`model.fit(x, y_true, batch_size=1, epochs=1500)`

- `x`:入力データ
- `y_true`:正しい答えとして期待しているラベル
- `epochs`:学習する回数
- `batch_size`:「バッチサイズ」は、入力データを何点を一緒にまとめて学習させるのか

In [7]:
# 学習データを用意
y_true = np.array([
    [0], 
    [0], 
    [0], 
    [1]
], dtype = float)


model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 [==============================] - 0s 3ms/step - loss: 0.2936
Epoch 2/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2934
Epoch 3/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2932
Epoch 4/1500
4/4 [==============================] - 0s 999us/step - loss: 0.2930
Epoch 5/1500
4/4 [==============================] - 0s 1000us/step - loss: 0.2928
Epoch 6/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2925
Epoch 7/1500
4/4 [==============================] - 0s 1000us/step - loss: 0.2923
Epoch 8/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2921
Epoch 9/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2919
Epoch 10/1500
4/4 [==============================] - 0s 999us/step - loss: 0.2917
Epoch 11/1500
4/4 [==============================] - 0s 1000us/step - loss: 0.2915
Epoch 12/1500
4/4 [==============================] - 0s 1000us/step - loss: 0.2913
Epoch 13/1500
4/4 [==============

4/4 [==============================] - 0s 0s/step - loss: 0.1753
Epoch 206/3000
4/4 [==============================] - 0s 4ms/step - loss: 0.1751
Epoch 207/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1749
Epoch 208/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1747
Epoch 209/3000
4/4 [==============================] - 0s 4ms/step - loss: 0.1746
Epoch 210/3000
4/4 [==============================] - 0s 649us/step - loss: 0.1744
Epoch 211/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1742
Epoch 212/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1740
Epoch 213/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1739
Epoch 214/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1737
Epoch 215/3000
4/4 [==============================] - 0s 4ms/step - loss: 0.1735
Epoch 216/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1734
Epoch 217/3000
4/4 [============================

4/4 [==============================] - 0s 0s/step - loss: 0.1599
Epoch 307/3000
4/4 [==============================] - 0s 498us/step - loss: 0.1598
Epoch 308/3000
4/4 [==============================] - 0s 175us/step - loss: 0.1596
Epoch 309/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1595
Epoch 310/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1594
Epoch 311/3000
4/4 [==============================] - 0s 565us/step - loss: 0.1592
Epoch 312/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1591
Epoch 313/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1590
Epoch 314/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1588
Epoch 315/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1587
Epoch 316/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1586
Epoch 317/3000
4/4 [==============================] - 0s 693us/step - loss: 0.1584
Epoch 318/3000
4/4 [=====================

4/4 [==============================] - 0s 999us/step - loss: 0.1476
Epoch 408/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1475
Epoch 409/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1474
Epoch 410/3000
4/4 [==============================] - 0s 748us/step - loss: 0.1473
Epoch 411/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1472
Epoch 412/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1471
Epoch 413/3000
4/4 [==============================] - 0s 749us/step - loss: 0.1470
Epoch 414/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1469
Epoch 415/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1468
Epoch 416/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1466
Epoch 417/3000
4/4 [==============================] - 0s 982us/step - loss: 0.1465
Epoch 418/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1464
Epoch 419/3000
4/4 [======================

4/4 [==============================] - 0s 398us/step - loss: 0.1373
Epoch 508/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1372
Epoch 509/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1371
Epoch 510/3000
4/4 [==============================] - 0s 653us/step - loss: 0.1370
Epoch 511/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1370
Epoch 512/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1369
Epoch 513/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1368
Epoch 514/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1367
Epoch 515/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1366
Epoch 516/3000
4/4 [==============================] - 0s 700us/step - loss: 0.1365
Epoch 517/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1364
Epoch 518/3000
4/4 [==============================] - 0s 0s/step - loss: 0.1363
Epoch 519/3000
4/4 [========================

...

4/4 [==============================] - 0s 1ms/step - loss: 0.0563
Epoch 2592/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0562
Epoch 2593/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0562
Epoch 2594/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0562
Epoch 2595/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0562
Epoch 2596/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0562
Epoch 2597/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0562
Epoch 2598/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0561
Epoch 2599/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0561
Epoch 2600/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0561
Epoch 2601/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0561
Epoch 2602/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0561
Epoch 2603/3000
4/4 [===========

4/4 [==============================] - 0s 1ms/step - loss: 0.0547
Epoch 2691/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0546
Epoch 2692/3000
4/4 [==============================] - 0s 748us/step - loss: 0.0546
Epoch 2693/3000
4/4 [==============================] - 0s 995us/step - loss: 0.0546
Epoch 2694/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0546
Epoch 2695/3000
4/4 [==============================] - 0s 999us/step - loss: 0.0546
Epoch 2696/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0546
Epoch 2697/3000
4/4 [==============================] - 0s 997us/step - loss: 0.0546
Epoch 2698/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0545
Epoch 2699/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0545
Epoch 2700/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0545
Epoch 2701/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0545
Epoch 2702/3000
4/4 [===

4/4 [==============================] - 0s 2ms/step - loss: 0.0531
Epoch 2790/3000
4/4 [==============================] - 0s 751us/step - loss: 0.0531
Epoch 2791/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0531
Epoch 2792/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 2793/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 2794/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 2795/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0531
Epoch 2796/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0530
Epoch 2797/3000
4/4 [==============================] - 0s 997us/step - loss: 0.0530
Epoch 2798/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0530
Epoch 2799/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0530
Epoch 2800/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0530
Epoch 2801/3000
4/4 [=======

4/4 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2889/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2890/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0517
Epoch 2891/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2892/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0516
Epoch 2893/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 2894/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 2895/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 2896/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 2897/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0516
Epoch 2898/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0516
Epoch 2899/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0515
Epoch 2900/3000
4/4 [===========

4/4 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 2988/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0503
Epoch 2989/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0503
Epoch 2990/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 2991/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0503
Epoch 2992/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 2993/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0502
Epoch 2994/3000
4/4 [==============================] - 0s 998us/step - loss: 0.0502
Epoch 2995/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0502
Epoch 2996/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0502
Epoch 2997/3000
4/4 [==============================] - 0s 996us/step - loss: 0.0502
Epoch 2998/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0502
Epoch 2999/3000
4/4 [=======

In [8]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 [==============================] - 0s 36ms/step
[[0.16937435]
 [0.32650128]
 [0.3279375 ]
 [0.5370538 ]]


## 改善方法：適する損失関数を選ぶ

「平均二乗誤差」普段に「距離」が表している（正解数値からどれぐらい離れるのか？）。実数の場合はこれで良いと思っていいのが、学習データはバイナリ（0か1）の場合、kerasがもと良い損失関数を提供する。それは`binary_crossentropy`という。

それでは、関数を変えて、確かめてみよう～

In [9]:
# もう一度乱数を固定、モデルを構築する
model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")

In [10]:
# 学習実行
model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7573
Epoch 2/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7546
Epoch 3/1500
4/4 [==============================] - 0s 668us/step - loss: 0.7520
Epoch 4/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7494
Epoch 5/1500
4/4 [==============================] - 0s 669us/step - loss: 0.7468
Epoch 6/1500
4/4 [==============================] - 0s 834us/step - loss: 0.7443
Epoch 7/1500
4/4 [==============================] - 0s 835us/step - loss: 0.7418
Epoch 8/1500
4/4 [==============================] - 0s 666us/step - loss: 0.7393
Epoch 9/1500
4/4 [==============================] - 0s 835us/step - loss: 0.7369
Epoch 10/1500
4/4 [==============================] - 0s 666us/step - loss: 0.7345
Epoch 11/1500
4/4 [==============================] - 0s 835us/step - loss: 0.7322
Epoch 12/1500
4/4 [==============================] - 0s 667us/step - loss: 0.7299
Epoch 13/1500
4/4 [============

4/4 [==============================] - 0s 790us/step - loss: 0.4536
Epoch 201/3000
4/4 [==============================] - 0s 996us/step - loss: 0.4531
Epoch 202/3000
4/4 [==============================] - 0s 500us/step - loss: 0.4526
Epoch 203/3000
4/4 [==============================] - 0s 638us/step - loss: 0.4521
Epoch 204/3000
4/4 [==============================] - 0s 498us/step - loss: 0.4516
Epoch 205/3000
4/4 [==============================] - 0s 600us/step - loss: 0.4511
Epoch 206/3000
4/4 [==============================] - 0s 749us/step - loss: 0.4506
Epoch 207/3000
4/4 [==============================] - 0s 862us/step - loss: 0.4501
Epoch 208/3000
4/4 [==============================] - 0s 774us/step - loss: 0.4496
Epoch 209/3000
4/4 [==============================] - 0s 624us/step - loss: 0.4491
Epoch 210/3000
4/4 [==============================] - 0s 776us/step - loss: 0.4486
Epoch 211/3000
4/4 [==============================] - 0s 998us/step - loss: 0.4481
Epoch 212/3000
4/4 

4/4 [==============================] - 0s 873us/step - loss: 0.4095
Epoch 300/3000
4/4 [==============================] - 0s 627us/step - loss: 0.4091
Epoch 301/3000
4/4 [==============================] - 0s 747us/step - loss: 0.4087
Epoch 302/3000
4/4 [==============================] - 0s 605us/step - loss: 0.4083
Epoch 303/3000
4/4 [==============================] - 0s 756us/step - loss: 0.4079
Epoch 304/3000
4/4 [==============================] - 0s 666us/step - loss: 0.4075
Epoch 305/3000
4/4 [==============================] - 0s 890us/step - loss: 0.4071
Epoch 306/3000
4/4 [==============================] - 0s 780us/step - loss: 0.4067
Epoch 307/3000
4/4 [==============================] - 0s 786us/step - loss: 0.4064
Epoch 308/3000
4/4 [==============================] - 0s 615us/step - loss: 0.4060
Epoch 309/3000
4/4 [==============================] - 0s 713us/step - loss: 0.4056
Epoch 310/3000
4/4 [==============================] - 0s 617us/step - loss: 0.4052
Epoch 311/3000
4/4 

4/4 [==============================] - 0s 756us/step - loss: 0.3741
Epoch 399/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3738
Epoch 400/3000
4/4 [==============================] - 0s 785us/step - loss: 0.3735
Epoch 401/3000
4/4 [==============================] - 0s 984us/step - loss: 0.3731
Epoch 402/3000
4/4 [==============================] - 0s 587us/step - loss: 0.3728
Epoch 403/3000
4/4 [==============================] - 0s 624us/step - loss: 0.3725
Epoch 404/3000
4/4 [==============================] - 0s 749us/step - loss: 0.3722
Epoch 405/3000
4/4 [==============================] - 0s 627us/step - loss: 0.3719
Epoch 406/3000
4/4 [==============================] - 0s 870us/step - loss: 0.3715
Epoch 407/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3712
Epoch 408/3000
4/4 [==============================] - 0s 498us/step - loss: 0.3709
Epoch 409/3000
4/4 [==============================] - 0s 620us/step - loss: 0.3706
Epoch 410/3000
4/4 [===

4/4 [==============================] - 0s 963us/step - loss: 0.3449
Epoch 498/3000
4/4 [==============================] - 0s 894us/step - loss: 0.3447
Epoch 499/3000
4/4 [==============================] - 0s 832us/step - loss: 0.3444
Epoch 500/3000
4/4 [==============================] - 0s 777us/step - loss: 0.3441
Epoch 501/3000
4/4 [==============================] - 0s 803us/step - loss: 0.3439
Epoch 502/3000
4/4 [==============================] - 0s 634us/step - loss: 0.3436
Epoch 503/3000
4/4 [==============================] - 0s 981us/step - loss: 0.3433
Epoch 504/3000
4/4 [==============================] - 0s 868us/step - loss: 0.3431
Epoch 505/3000
4/4 [==============================] - 0s 862us/step - loss: 0.3428
Epoch 506/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3425
Epoch 507/3000
4/4 [==============================] - 0s 633us/step - loss: 0.3423
Epoch 508/3000
4/4 [==============================] - 0s 938us/step - loss: 0.3420
Epoch 509/3000
4/4 [=

...

Epoch 2588/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1371
Epoch 2589/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1370
Epoch 2590/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1370
Epoch 2591/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1369
Epoch 2592/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1369
Epoch 2593/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1369
Epoch 2594/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1368
Epoch 2595/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1368
Epoch 2596/3000
4/4 [==============================] - 0s 998us/step - loss: 0.1367
Epoch 2597/3000
4/4 [==============================] - 0s 998us/step - loss: 0.1367
Epoch 2598/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1367
Epoch 2599/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1366
Epoch 2600/3

4/4 [==============================] - 0s 1ms/step - loss: 0.1333
Epoch 2688/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1332
Epoch 2689/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1332
Epoch 2690/3000
4/4 [==============================] - 0s 904us/step - loss: 0.1332
Epoch 2691/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1331
Epoch 2692/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1331
Epoch 2693/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1331
Epoch 2694/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1330
Epoch 2695/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1330
Epoch 2696/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1330
Epoch 2697/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1329
Epoch 2698/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1329
Epoch 2699/3000
4/4 [=========

Epoch 2787/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1297
Epoch 2788/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1296
Epoch 2789/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1296
Epoch 2790/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1296
Epoch 2791/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1295
Epoch 2792/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1295
Epoch 2793/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1295
Epoch 2794/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1294
Epoch 2795/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1294
Epoch 2796/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1294
Epoch 2797/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1293
Epoch 2798/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1293
Epoch 2799/3000


4/4 [==============================] - 0s 2ms/step - loss: 0.1263
Epoch 2887/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1263
Epoch 2888/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1262
Epoch 2889/3000
4/4 [==============================] - 0s 893us/step - loss: 0.1262
Epoch 2890/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1262
Epoch 2891/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1261
Epoch 2892/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1261
Epoch 2893/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1261
Epoch 2894/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1260
Epoch 2895/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1260
Epoch 2896/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1260
Epoch 2897/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1259
Epoch 2898/3000
4/4 [=========

4/4 [==============================] - 0s 877us/step - loss: 0.1231
Epoch 2986/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1230
Epoch 2987/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1230
Epoch 2988/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1230
Epoch 2989/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1229
Epoch 2990/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1229
Epoch 2991/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1229
Epoch 2992/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1229
Epoch 2993/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1228
Epoch 2994/3000
4/4 [==============================] - 0s 998us/step - loss: 0.1228
Epoch 2995/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1228
Epoch 2996/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1227
Epoch 2997/3000
4/4 [=======

In [11]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 [==============================] - 0s 34ms/step
[[0.02484997]
 [0.2086259 ]
 [0.19051076]
 [0.70885134]]


In [12]:
print(model.get_weights())

[array([[2.2230399],
       [2.3365066]], dtype=float32), array([-3.6697347], dtype=float32)]


## 練習：ORを学習させる

In [13]:
# 学習データを用意
y_true = np.array([
    [0], 
    [1], 
    [1], 
    [1]
], dtype = float)

model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")
model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.3682
Epoch 2/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.3673
Epoch 3/1500
4/4 [==============================] - 0s 834us/step - loss: 0.3664
Epoch 4/1500
4/4 [==============================] - 0s 667us/step - loss: 0.3656
Epoch 5/1500
4/4 [==============================] - 0s 666us/step - loss: 0.3648
Epoch 6/1500
4/4 [==============================] - 0s 668us/step - loss: 0.3639
Epoch 7/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.3631
Epoch 8/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.3623
Epoch 9/1500
4/4 [==============================] - 0s 667us/step - loss: 0.3615
Epoch 10/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.3607
Epoch 11/1500
4/4 [==============================] - 0s 667us/step - loss: 0.3599
Epoch 12/1500
4/4 [==============================] - 0s 669us/step - loss: 0.3592
Epoch 13/1500
4/4 [================

4/4 [==============================] - 0s 996us/step - loss: 0.2712
Epoch 201/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2710
Epoch 202/3000
4/4 [==============================] - 0s 747us/step - loss: 0.2707
Epoch 203/3000
4/4 [==============================] - 0s 748us/step - loss: 0.2704
Epoch 204/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2702
Epoch 205/3000
4/4 [==============================] - 0s 490us/step - loss: 0.2699
Epoch 206/3000
4/4 [==============================] - 0s 995us/step - loss: 0.2697
Epoch 207/3000
4/4 [==============================] - 0s 767us/step - loss: 0.2694
Epoch 208/3000
4/4 [==============================] - 0s 730us/step - loss: 0.2692
Epoch 209/3000
4/4 [==============================] - 0s 996us/step - loss: 0.2690
Epoch 210/3000
4/4 [==============================] - 0s 996us/step - loss: 0.2687
Epoch 211/3000
4/4 [==============================] - 0s 485us/step - loss: 0.2685
Epoch 212/3000
4/4 [===

4/4 [==============================] - 0s 748us/step - loss: 0.2484
Epoch 300/3000
4/4 [==============================] - 0s 497us/step - loss: 0.2482
Epoch 301/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2480
Epoch 302/3000
4/4 [==============================] - 0s 738us/step - loss: 0.2478
Epoch 303/3000
4/4 [==============================] - 0s 748us/step - loss: 0.2476
Epoch 304/3000
4/4 [==============================] - 0s 957us/step - loss: 0.2474
Epoch 305/3000
4/4 [==============================] - 0s 748us/step - loss: 0.2472
Epoch 306/3000
4/4 [==============================] - 0s 738us/step - loss: 0.2469
Epoch 307/3000
4/4 [==============================] - 0s 750us/step - loss: 0.2467
Epoch 308/3000
4/4 [==============================] - 0s 645us/step - loss: 0.2465
Epoch 309/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.2463
Epoch 310/3000
4/4 [==============================] - 0s 748us/step - loss: 0.2461
Epoch 311/3000
4/4 [===

4/4 [==============================] - 0s 508us/step - loss: 0.2291
Epoch 399/3000
4/4 [==============================] - 0s 764us/step - loss: 0.2289
Epoch 400/3000
4/4 [==============================] - 0s 483us/step - loss: 0.2287
Epoch 401/3000
4/4 [==============================] - 0s 997us/step - loss: 0.2286
Epoch 402/3000
4/4 [==============================] - 0s 747us/step - loss: 0.2284
Epoch 403/3000
4/4 [==============================] - 0s 733us/step - loss: 0.2282
Epoch 404/3000
4/4 [==============================] - 0s 996us/step - loss: 0.2280
Epoch 405/3000
4/4 [==============================] - 0s 750us/step - loss: 0.2278
Epoch 406/3000
4/4 [==============================] - 0s 748us/step - loss: 0.2277
Epoch 407/3000
4/4 [==============================] - 0s 998us/step - loss: 0.2275
Epoch 408/3000
4/4 [==============================] - 0s 748us/step - loss: 0.2273
Epoch 409/3000
4/4 [==============================] - 0s 995us/step - loss: 0.2271
Epoch 410/3000
4/4 

4/4 [==============================] - 0s 748us/step - loss: 0.2124
Epoch 498/3000
4/4 [==============================] - 0s 996us/step - loss: 0.2123
Epoch 499/3000
4/4 [==============================] - 0s 498us/step - loss: 0.2121
Epoch 500/3000
4/4 [==============================] - 0s 748us/step - loss: 0.2120
Epoch 501/3000
4/4 [==============================] - 0s 747us/step - loss: 0.2118
Epoch 502/3000
4/4 [==============================] - 0s 737us/step - loss: 0.2116
Epoch 503/3000
4/4 [==============================] - 0s 987us/step - loss: 0.2115
Epoch 504/3000
4/4 [==============================] - 0s 510us/step - loss: 0.2113
Epoch 505/3000
4/4 [==============================] - 0s 747us/step - loss: 0.2112
Epoch 506/3000
4/4 [==============================] - 0s 994us/step - loss: 0.2110
Epoch 507/3000
4/4 [==============================] - 0s 887us/step - loss: 0.2109
Epoch 508/3000
4/4 [==============================] - 0s 742us/step - loss: 0.2107
Epoch 509/3000
4/4 

...

Epoch 2587/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0796
Epoch 2588/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0795
Epoch 2589/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0795
Epoch 2590/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0795
Epoch 2591/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0795
Epoch 2592/3000
4/4 [==============================] - 0s 765us/step - loss: 0.0794
Epoch 2593/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0794
Epoch 2594/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0794
Epoch 2595/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0794
Epoch 2596/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0793
Epoch 2597/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0793
Epoch 2598/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0793
Epoch 2599/300

4/4 [==============================] - 0s 2ms/step - loss: 0.0772
Epoch 2687/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0771
Epoch 2688/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0771
Epoch 2689/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0771
Epoch 2690/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0771
Epoch 2691/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0770
Epoch 2692/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0770
Epoch 2693/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0770
Epoch 2694/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0770
Epoch 2695/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0769
Epoch 2696/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0769
Epoch 2697/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0769
Epoch 2698/3000
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.0749
Epoch 2787/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0748
Epoch 2788/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0748
Epoch 2789/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0748
Epoch 2790/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0748
Epoch 2791/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0747
Epoch 2792/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0747
Epoch 2793/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0747
Epoch 2794/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0747
Epoch 2795/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0747
Epoch 2796/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0746
Epoch 2797/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0746
Epoch 2798/3000
4/4 [===========

Epoch 2886/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0727
Epoch 2887/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0727
Epoch 2888/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0726
Epoch 2889/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0726
Epoch 2890/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0726
Epoch 2891/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0726
Epoch 2892/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0726
Epoch 2893/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0725
Epoch 2894/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0725
Epoch 2895/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0725
Epoch 2896/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0725
Epoch 2897/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0725
Epoch 2898/3000


4/4 [==============================] - 0s 2ms/step - loss: 0.0707
Epoch 2986/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0706
Epoch 2987/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0706
Epoch 2988/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0706
Epoch 2989/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0706
Epoch 2990/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0706
Epoch 2991/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0705
Epoch 2992/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0705
Epoch 2993/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0705
Epoch 2994/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0705
Epoch 2995/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0705
Epoch 2996/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0704
Epoch 2997/3000
4/4 [===========

In [14]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 [==============================] - 0s 33ms/step
[[0.24802642]
 [0.90814644]
 [0.9033938 ]
 [0.9964452 ]]


In [15]:
# 加重を確認
print(model.get_weights())

[array([[3.3446815],
       [3.400376 ]], dtype=float32), array([-1.1091659], dtype=float32)]


## 練習：XORを学習させる

In [16]:
# 学習データを用意
y_true = np.array([
    [0], 
    [1], 
    [1], 
    [0]
], dtype = float)

model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")
model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7203
Epoch 2/1500
4/4 [==============================] - 0s 1000us/step - loss: 0.7197
Epoch 3/1500
4/4 [==============================] - 0s 668us/step - loss: 0.7191
Epoch 4/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7184
Epoch 5/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7178
Epoch 6/1500
4/4 [==============================] - 0s 667us/step - loss: 0.7172
Epoch 7/1500
4/4 [==============================] - 0s 835us/step - loss: 0.7167
Epoch 8/1500
4/4 [==============================] - 0s 667us/step - loss: 0.7161
Epoch 9/1500
4/4 [==============================] - 0s 834us/step - loss: 0.7156
Epoch 10/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7151
Epoch 11/1500
4/4 [==============================] - 0s 667us/step - loss: 0.7146
Epoch 12/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.7141
Epoch 13/1500
4/4 [===============

4/4 [==============================] - 0s 934us/step - loss: 0.6967
Epoch 201/3000
4/4 [==============================] - 0s 747us/step - loss: 0.6967
Epoch 202/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6967
Epoch 203/3000
4/4 [==============================] - 0s 513us/step - loss: 0.6967
Epoch 204/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6967
Epoch 205/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6967
Epoch 206/3000
4/4 [==============================] - 0s 747us/step - loss: 0.6967
Epoch 207/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6967
Epoch 208/3000
4/4 [==============================] - 0s 658us/step - loss: 0.6967
Epoch 209/3000
4/4 [==============================] - 0s 748us/step - loss: 0.6967
Epoch 210/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6967
Epoch 211/3000
4/4 [==============================] - 0s 501us/step - loss: 0.6967
Epoch 212/3000
4/4 [=========

4/4 [==============================] - 0s 880us/step - loss: 0.6964
Epoch 300/3000
4/4 [==============================] - 0s 482us/step - loss: 0.6964
Epoch 301/3000
4/4 [==============================] - 0s 878us/step - loss: 0.6964
Epoch 302/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6964
Epoch 303/3000
4/4 [==============================] - 0s 606us/step - loss: 0.6964
Epoch 304/3000
4/4 [==============================] - 0s 748us/step - loss: 0.6964
Epoch 305/3000
4/4 [==============================] - 0s 753us/step - loss: 0.6964
Epoch 306/3000
4/4 [==============================] - 0s 498us/step - loss: 0.6963
Epoch 307/3000
4/4 [==============================] - 0s 822us/step - loss: 0.6963
Epoch 308/3000
4/4 [==============================] - 0s 999us/step - loss: 0.6963
Epoch 309/3000
4/4 [==============================] - 0s 626us/step - loss: 0.6963
Epoch 310/3000
4/4 [==============================] - 0s 746us/step - loss: 0.6963
Epoch 311/3000
4/4 [=

4/4 [==============================] - 0s 997us/step - loss: 0.6961
Epoch 399/3000
4/4 [==============================] - 0s 749us/step - loss: 0.6961
Epoch 400/3000
4/4 [==============================] - 0s 874us/step - loss: 0.6961
Epoch 401/3000
4/4 [==============================] - 0s 748us/step - loss: 0.6961
Epoch 402/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6961
Epoch 403/3000
4/4 [==============================] - 0s 747us/step - loss: 0.6961
Epoch 404/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6961
Epoch 405/3000
4/4 [==============================] - 0s 499us/step - loss: 0.6961
Epoch 406/3000
4/4 [==============================] - 0s 587us/step - loss: 0.6961
Epoch 407/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6961
Epoch 408/3000
4/4 [==============================] - 0s 747us/step - loss: 0.6961
Epoch 409/3000
4/4 [==============================] - 0s 989us/step - loss: 0.6961
Epoch 410/3000
4/4 [=====

4/4 [==============================] - 0s 1ms/step - loss: 0.6960
Epoch 498/3000
4/4 [==============================] - 0s 748us/step - loss: 0.6960
Epoch 499/3000
4/4 [==============================] - 0s 747us/step - loss: 0.6960
Epoch 500/3000
4/4 [==============================] - 0s 504us/step - loss: 0.6960
Epoch 501/3000
4/4 [==============================] - 0s 748us/step - loss: 0.6960
Epoch 502/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6960
Epoch 503/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6960
Epoch 504/3000
4/4 [==============================] - 0s 733us/step - loss: 0.6960
Epoch 505/3000
4/4 [==============================] - 0s 630us/step - loss: 0.6960
Epoch 506/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.6960
Epoch 507/3000
4/4 [==============================] - 0s 843us/step - loss: 0.6960
Epoch 508/3000
4/4 [==============================] - 0s 499us/step - loss: 0.6960
Epoch 509/3000
4/4 [=======

...

4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2598/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2599/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2600/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2601/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2602/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2603/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2604/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2605/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2606/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2607/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2608/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2609/3000
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2698/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2699/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2700/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2701/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2702/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2703/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2704/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2705/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2706/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2707/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6957
Epoch 2708/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6957
Epoch 2709/3000
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2798/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2799/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2800/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2801/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2802/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6956
Epoch 2803/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6957
Epoch 2804/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6957
Epoch 2805/3000
4/4 [==============================] - 0s 4ms/step - loss: 0.6957
Epoch 2806/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6956
Epoch 2807/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6957
Epoch 2808/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2809/3000
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2898/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2899/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2900/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2901/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2902/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2903/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6956
Epoch 2904/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6957
Epoch 2905/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2906/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2907/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.6957
Epoch 2908/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2909/3000
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 2998/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957
Epoch 2999/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6956
Epoch 3000/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.6957


In [17]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 [==============================] - 0s 40ms/step
[[0.49457237]
 [0.49908355]
 [0.49921224]
 [0.50372356]]
